### **Libraries**

In [13]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np

In [16]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### **Dataset**

The **CIFAR-100** dataset consists of of 60,000 color images, each of size 32×32 pixels, divided into 100 classes with 600 images per class. The dataset is further split into 50,000 training images and 10,000 test images.

In this case, images are resized to match the required input size of some models as most of them require a 225x225 image size as they were programmed on ImageNet.

In [17]:
# Size tranformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match models' expected input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet mean values
])

# Size tranformation for InceptionV3
transform_inception = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize to 299x299 pixels for InceptionV3
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# CIFAR-100 dataset for InceptionV3
trainset_inception = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_inception)
trainloader_inception = DataLoader(trainset_inception, batch_size=32, shuffle=True, num_workers=2)

testset_inception = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_inception)
testloader_inception = DataLoader(testset_inception, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


The CNN architectures that will be considered are the following:

- **AlexNet:** Original architecture
- **ResNet:** ResNet-50
- **VGGNet:** VGG-16
- **GoogleNet:** Inception v3
- **EfficientNet:** EfficientNet-B1, EfficientNet-B3 EfficientNet-B5

In [18]:
# Define models
models = {
    'AlexNet': torchvision.models.alexnet(pretrained=True),
    'ResNet50': torchvision.models.resnet50(pretrained=True),
    'VGG16': torchvision.models.vgg16(pretrained=True),
    'InceptionV3': torchvision.models.inception_v3(pretrained=True, aux_logits=True),
    'EfficientNet-B1': torchvision.models.efficientnet_b1(pretrained=True),
    'EfficientNet-B3': torchvision.models.efficientnet_b3(pretrained=True),
    'EfficientNet-B5': torchvision.models.efficientnet_b5(pretrained=True)
}

c:\Users\aleca\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aleca\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\aleca/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:10<00:00, 24.2MB/s] 
c:\Users\aleca\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWa

The following code ensures that each model is adapted to the CIFAR-100 dataset by replacing the final layer of the model. For models like AlexNet, VGG, and EfficientNet, this modification is made to the classifier's last layer, while for Inception and other models, it's made to the last fully connected layer 

In [19]:
# Modify the final layer to match CIFAR-100 classes
for name, model in models.items():
    if 'EfficientNet' in name:
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, 100)
    elif 'Inception' in name:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 100)
    elif 'AlexNet' in name or 'VGG' in name:
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, 100)
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 100)
    models[name] = model.to(device)

This part initializes the loss function used for training the models. Cross-entropy loss measures how well a model's predicted probability distribution matches the actual distribution of the labels.

The mathematical definition of the cross-entropy loss for a dataset is defined as:

$$
L = -\frac{1}{N} \sum_{n=1}^N \sum_{i=1}^C y_{ni} \log(p_{ni})
$$

- \(N\): The number of samples.
- \(y_{ni}\): The actual binary indicator for sample \(n\) and class \(i\).
- \(p_{ni}\): The predicted probability for sample \(n\) and class \(i\).

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()

For the following part:

The **train_model** funtion trains a given model on the training dataset for a specified number of epochs while tracking the loss and accuracy.

The **evaluate_model** functio evaluates the trained model on the test dataset.

The **train_model_inception** function trains the InceptionV3 model with accuracy tracking. The function outputs a tuple when *aux_logits* is *True*. The code checks if the output is a tuple and uses only the main output (ignoring auxiliary outputs).

In [20]:
# Function to train the model with accuracy tracking
def train_model(model, trainloader, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        start_time = time.time()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)

        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = correct / total
        end_time = time.time()

        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Time: {end_time - start_time:.2f} seconds")

# Function to evaluate the model
def evaluate_model(model, testloader):
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

# Define a modified train function for InceptionV3
def train_model_inception(model, trainloader, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        start_time = time.time()
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # Use only the main output
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        end_time = time.time()
        epoch_accuracy = correct / total
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}, Accuracy: {epoch_accuracy * 100:.2f}%, Time: {end_time - start_time} seconds")

### **Model's training**

Now, each one of the architectures previously mentioned are trained in the same way. The models are trained in the training data for 25 epochs using the Adam optimizer with a learning rate of 0.001. The most important definition of this section are the following:

- **Epoch** refers to one complete pass of the entire training dataset through the learning algorithm. 

- **Adam optimizer** (*optim.Adam*) is an adaptive learning rate optimization algorithm designed specifically for training deep neural networks. It combines the advantages of two other extensions of stochastic gradient descent: AdaGrad and RMSProp.

- **Learning rate** controls how much to change the model in response to the estimated error each time the model weights are updated during training. 

After training, the model is evaluated on the test data (testloader) to calculate the accuracy and F1 score. Finally, the results including the accuracy, F1 score, and training time are printed to the console.

#### **AlexNet**

In [ ]:
# Train and evaluate AlexNet
model_name = 'AlexNet'
print(f"\nTraining {model_name}...\n")
model = models[model_name]
optimizer = optim.Adam(model.parameters(), lr=0.001) # lr=0.001 is considered standar for the learning rate
start_time = time.time()
train_model(model, trainloader, criterion, optimizer, num_epochs=25) # num_epochs=25 due to hardware limitation
training_time = time.time() - start_time
accuracy, f1 = evaluate_model(model, testloader)

print(f"{model_name} - Top-1 Accuracy: {accuracy * 100:.2f}%")
print(f"{model_name} - F1 Score: {f1:.2f}")
print(f"{model_name} - Training Time: {training_time:.2f} seconds")


Training AlexNet...

Epoch 1, Loss: 4.6061, Accuracy: 0.0085, Time: 61.72 seconds
Epoch 2, Loss: 4.6061, Accuracy: 0.0094, Time: 61.00 seconds
Epoch 3, Loss: 4.6060, Accuracy: 0.0090, Time: 60.95 seconds
Epoch 4, Loss: 4.6060, Accuracy: 0.0092, Time: 60.88 seconds
Epoch 5, Loss: 4.6060, Accuracy: 0.0083, Time: 61.06 seconds
Epoch 6, Loss: 4.6061, Accuracy: 0.0093, Time: 60.86 seconds
Epoch 7, Loss: 4.6061, Accuracy: 0.0086, Time: 60.83 seconds
Epoch 8, Loss: 4.6060, Accuracy: 0.0089, Time: 61.31 seconds
Epoch 9, Loss: 4.6060, Accuracy: 0.0094, Time: 61.92 seconds
Epoch 10, Loss: 4.6060, Accuracy: 0.0090, Time: 61.70 seconds
Epoch 11, Loss: 4.6060, Accuracy: 0.0088, Time: 61.79 seconds
Epoch 12, Loss: 4.6060, Accuracy: 0.0086, Time: 61.68 seconds
Epoch 13, Loss: 4.6060, Accuracy: 0.0086, Time: 61.60 seconds
Epoch 14, Loss: 4.6059, Accuracy: 0.0085, Time: 61.79 seconds
Epoch 15, Loss: 4.6059, Accuracy: 0.0089, Time: 61.83 seconds
Epoch 16, Loss: 4.6059, Accuracy: 0.0087, Time: 61.64 sec

#### **ResNet-50**

In [ ]:
# Train and evaluate ResNet50
model_name = 'ResNet50'
print(f"\nTraining {model_name}...\n")
model = models[model_name]
optimizer = optim.Adam(model.parameters(), lr=0.001)
start_time = time.time()
train_model(model, trainloader, criterion, optimizer, num_epochs=25)
training_time = time.time() - start_time
accuracy, f1 = evaluate_model(model, testloader)

print(f"{model_name} - Top-1 Accuracy: {accuracy * 100:.2f}%")
print(f"{model_name} - F1 Score: {f1:.2f}")
print(f"{model_name} - Training Time: {training_time:.2f} seconds")


Training ResNet50...

Epoch 1, Loss: 3.8320, Accuracy: 0.1001, Time: 239.96 seconds
Epoch 2, Loss: 2.9230, Accuracy: 0.2579, Time: 239.93 seconds
Epoch 3, Loss: 2.3188, Accuracy: 0.3825, Time: 239.56 seconds
Epoch 4, Loss: 1.9734, Accuracy: 0.4613, Time: 238.06 seconds
Epoch 5, Loss: 1.7027, Accuracy: 0.5245, Time: 238.09 seconds
Epoch 6, Loss: 1.4844, Accuracy: 0.5799, Time: 237.72 seconds
Epoch 7, Loss: 1.2972, Accuracy: 0.6244, Time: 237.89 seconds
Epoch 8, Loss: 1.1180, Accuracy: 0.6698, Time: 238.16 seconds
Epoch 9, Loss: 0.9522, Accuracy: 0.7152, Time: 238.42 seconds
Epoch 10, Loss: 0.7862, Accuracy: 0.7604, Time: 238.61 seconds
Epoch 11, Loss: 0.6475, Accuracy: 0.7972, Time: 238.55 seconds
Epoch 12, Loss: 0.5247, Accuracy: 0.8347, Time: 238.55 seconds
Epoch 13, Loss: 0.4255, Accuracy: 0.8654, Time: 238.72 seconds
Epoch 14, Loss: 0.3589, Accuracy: 0.8851, Time: 238.45 seconds
Epoch 15, Loss: 0.3063, Accuracy: 0.9012, Time: 238.07 seconds
Epoch 16, Loss: 0.2609, Accuracy: 0.9160,

#### **VGG-16**

In [ ]:
# Train and evaluate VGG16
model_name = 'VGG16'
print(f"\nTraining {model_name}...\n")
model = models[model_name]
optimizer = optim.Adam(model.parameters(), lr=0.001)
start_time = time.time()
train_model(model, trainloader, criterion, optimizer, num_epochs=25)
training_time = time.time() - start_time
accuracy, f1 = evaluate_model(model, testloader)

print(f"{model_name} - Top-1 Accuracy: {accuracy * 100:.2f}%")
print(f"{model_name} - F1 Score: {f1:.2f}")
print(f"{model_name} - Training Time: {training_time:.2f} seconds")


Training VGG16...

Epoch 1, Loss: 4.6152, Accuracy: 0.0103, Time: 433.20 seconds
Epoch 2, Loss: 4.6086, Accuracy: 0.0102, Time: 430.03 seconds
Epoch 3, Loss: 4.6086, Accuracy: 0.0099, Time: 429.50 seconds
Epoch 4, Loss: 4.6077, Accuracy: 0.0100, Time: 429.51 seconds
Epoch 5, Loss: 4.6072, Accuracy: 0.0100, Time: 429.60 seconds
Epoch 6, Loss: 4.6074, Accuracy: 0.0091, Time: 430.41 seconds
Epoch 7, Loss: 4.6073, Accuracy: 0.0091, Time: 429.75 seconds
Epoch 8, Loss: 4.6075, Accuracy: 0.0091, Time: 429.76 seconds
Epoch 9, Loss: 4.6073, Accuracy: 0.0089, Time: 429.70 seconds
Epoch 10, Loss: 4.6070, Accuracy: 0.0091, Time: 430.45 seconds
Epoch 11, Loss: 4.6068, Accuracy: 0.0092, Time: 429.73 seconds
Epoch 12, Loss: 4.6068, Accuracy: 0.0098, Time: 429.73 seconds
Epoch 13, Loss: 4.6068, Accuracy: 0.0101, Time: 430.12 seconds
Epoch 14, Loss: 4.6069, Accuracy: 0.0095, Time: 429.81 seconds
Epoch 15, Loss: 4.6068, Accuracy: 0.0091, Time: 430.41 seconds
Epoch 16, Loss: 4.6066, Accuracy: 0.0096, Ti

#### **GoogleNet: Inception V3**

In [ ]:
# Training and evaluating InceptionV3
print("\nTraining InceptionV3...\n")
model_inceptionv3 = models['InceptionV3']
optimizer_inceptionv3 = optim.Adam(model_inceptionv3.parameters(), lr=0.001)

start_time = time.time()
train_model_inception(model_inceptionv3, trainloader_inception, criterion, optimizer_inceptionv3, num_epochs=25)
end_time = time.time()

accuracy_inceptionv3, f1_inceptionv3 = evaluate_model(model_inceptionv3, testloader_inception)
print(f"InceptionV3 - Top-1 Accuracy: {accuracy_inceptionv3 * 100:.2f}%")
print(f"InceptionV3 - F1 Score: {f1_inceptionv3:.2f}")
print(f"Training and evaluation time for InceptionV3: {end_time - start_time} seconds")


Training InceptionV3...

Epoch 1, Loss: 3.707519869581675, Accuracy: 11.49%, Time: 300.349223613739 seconds
Epoch 2, Loss: 2.5627949864759096, Accuracy: 31.46%, Time: 299.13457131385803 seconds
Epoch 3, Loss: 2.0100145956223696, Accuracy: 44.24%, Time: 299.8215389251709 seconds
Epoch 4, Loss: 1.6813800657931919, Accuracy: 52.73%, Time: 299.7049252986908 seconds
Epoch 5, Loss: 1.4426599984663233, Accuracy: 58.70%, Time: 299.4903144836426 seconds
Epoch 6, Loss: 1.2384110827211074, Accuracy: 63.61%, Time: 299.2987496852875 seconds
Epoch 7, Loss: 1.068663236066003, Accuracy: 68.38%, Time: 299.3534474372864 seconds
Epoch 8, Loss: 0.9201094804668914, Accuracy: 72.08%, Time: 299.6202931404114 seconds
Epoch 9, Loss: 0.7876180445183109, Accuracy: 75.63%, Time: 300.0845775604248 seconds
Epoch 10, Loss: 0.6714873101493142, Accuracy: 79.07%, Time: 299.27029252052307 seconds
Epoch 11, Loss: 0.5717141309458708, Accuracy: 82.16%, Time: 299.51088523864746 seconds
Epoch 12, Loss: 0.4924505337910704, A

#### **EfficientNet-B1**

In [ ]:
# Training and evaluating EfficientNet-B1
print("\nTraining EfficientNet-B1...\n")
model_efficientnet_b1 = models['EfficientNet-B1'].to(device)  # Ensure the model is on GPU
optimizer_efficientnet_b1 = optim.Adam(model_efficientnet_b1.parameters(), lr=0.001)

start_time = time.time()
train_model(model_efficientnet_b1, trainloader, criterion, optimizer_efficientnet_b1, num_epochs=25)
training_time = time.time() - start_time

accuracy_efficientnet_b1, f1_efficientnet_b1 = evaluate_model(model_efficientnet_b1, testloader)
print(f"EfficientNet-B1 - Top-1 Accuracy: {accuracy_efficientnet_b1 * 100:.2f}%")
print(f"EfficientNet-B1 - F1 Score: {f1_efficientnet_b1:.2f}")
print(f"EfficientNet-B1 - Training Time: {training_time:.2f} seconds")


Training EfficientNet-B1...

Epoch 1, Loss: 1.5150, Accuracy: 0.5959, Time: 207.93 seconds
Epoch 2, Loss: 0.8722, Accuracy: 0.7428, Time: 207.28 seconds
Epoch 3, Loss: 0.6783, Accuracy: 0.7948, Time: 207.19 seconds
Epoch 4, Loss: 0.5565, Accuracy: 0.8276, Time: 207.48 seconds
Epoch 5, Loss: 0.4617, Accuracy: 0.8551, Time: 207.55 seconds
Epoch 6, Loss: 0.3968, Accuracy: 0.8727, Time: 207.26 seconds
Epoch 7, Loss: 0.3453, Accuracy: 0.8895, Time: 207.39 seconds
Epoch 8, Loss: 0.2952, Accuracy: 0.9053, Time: 207.24 seconds
Epoch 9, Loss: 0.2598, Accuracy: 0.9168, Time: 207.19 seconds
Epoch 10, Loss: 0.2433, Accuracy: 0.9220, Time: 207.54 seconds
Epoch 11, Loss: 0.2167, Accuracy: 0.9293, Time: 207.40 seconds
Epoch 12, Loss: 0.2017, Accuracy: 0.9356, Time: 207.43 seconds
Epoch 13, Loss: 0.1896, Accuracy: 0.9392, Time: 207.55 seconds
Epoch 14, Loss: 0.1724, Accuracy: 0.9441, Time: 207.58 seconds
Epoch 15, Loss: 0.1693, Accuracy: 0.9455, Time: 207.23 seconds
Epoch 16, Loss: 0.1551, Accuracy: 

#### **EfficientNet-B3**

In [8]:
# Training and evaluating EfficientNet-B3
print("\nTraining EfficientNet-B3...\n")
model_efficientnet_b3 = models['EfficientNet-B3'].to(device)  # Ensure the model is on GPU
optimizer_efficientnet_b3 = optim.Adam(model_efficientnet_b3.parameters(), lr=0.001)

start_time = time.time()
train_model(model_efficientnet_b3, trainloader, criterion, optimizer_efficientnet_b3, num_epochs=25)
training_time = time.time() - start_time

accuracy_efficientnet_b3, f1_efficientnet_b3 = evaluate_model(model_efficientnet_b3, testloader)
print(f"EfficientNet-B3 - Top-1 Accuracy: {accuracy_efficientnet_b3 * 100:.2f}%")
print(f"EfficientNet-B3 - F1 Score: {f1_efficientnet_b3:.2f}")
print(f"EfficientNet-B3 - Training Time: {training_time:.2f} seconds")


Training EfficientNet-B3...

Epoch 1, Loss: 1.3788, Accuracy: 0.6197, Time: 137.48 seconds
Epoch 2, Loss: 0.8050, Accuracy: 0.7603, Time: 134.64 seconds
Epoch 3, Loss: 0.6326, Accuracy: 0.8058, Time: 135.59 seconds
Epoch 4, Loss: 0.5080, Accuracy: 0.8412, Time: 135.17 seconds
Epoch 5, Loss: 0.4236, Accuracy: 0.8670, Time: 135.59 seconds
Epoch 6, Loss: 0.3524, Accuracy: 0.8894, Time: 134.73 seconds
Epoch 7, Loss: 0.3079, Accuracy: 0.9018, Time: 135.25 seconds
Epoch 8, Loss: 0.2768, Accuracy: 0.9124, Time: 135.50 seconds
Epoch 9, Loss: 0.2455, Accuracy: 0.9226, Time: 135.46 seconds
Epoch 10, Loss: 0.2201, Accuracy: 0.9298, Time: 135.81 seconds
Epoch 11, Loss: 0.1991, Accuracy: 0.9355, Time: 135.59 seconds
Epoch 12, Loss: 0.1906, Accuracy: 0.9395, Time: 135.26 seconds
Epoch 13, Loss: 0.1738, Accuracy: 0.9451, Time: 135.31 seconds
Epoch 14, Loss: 0.1653, Accuracy: 0.9473, Time: 135.02 seconds
Epoch 15, Loss: 0.1550, Accuracy: 0.9498, Time: 135.98 seconds
Epoch 16, Loss: 0.1482, Accuracy: 

#### **EfficientNet-B5**

In [11]:
# Training and evaluating EfficientNet-B5
print("\nTraining EfficientNet-B5...\n")
model_efficientnet_b5 = models['EfficientNet-B5'].to(device)  # Ensure the model is on GPU
optimizer_efficientnet_b5 = optim.Adam(model_efficientnet_b5.parameters(), lr=0.001)

start_time = time.time()
train_model(model_efficientnet_b5, trainloader, criterion, optimizer_efficientnet_b5, num_epochs=25)
training_time = time.time() - start_time

accuracy_efficientnet_b5, f1_efficientnet_b5 = evaluate_model(model_efficientnet_b5, testloader)
print(f"EfficientNet-B5 - Top-1 Accuracy: {accuracy_efficientnet_b5 * 100:.2f}%")
print(f"EfficientNet-B5 - F1 Score: {f1_efficientnet_b5:.2f}")
print(f"EfficientNet-B5 - Training Time: {training_time:.2f} seconds")



Training EfficientNet-B5...

Epoch 1, Loss: 1.5030, Accuracy: 0.5906, Time: 213.76 seconds
Epoch 2, Loss: 0.9167, Accuracy: 0.7314, Time: 212.80 seconds
Epoch 3, Loss: 0.7039, Accuracy: 0.7888, Time: 213.30 seconds
Epoch 4, Loss: 0.5699, Accuracy: 0.8260, Time: 213.21 seconds
Epoch 5, Loss: 0.4699, Accuracy: 0.8553, Time: 213.38 seconds
Epoch 6, Loss: 0.3989, Accuracy: 0.8751, Time: 213.83 seconds
Epoch 7, Loss: 0.3424, Accuracy: 0.8919, Time: 212.98 seconds
Epoch 8, Loss: 0.2933, Accuracy: 0.9076, Time: 213.07 seconds
Epoch 9, Loss: 0.2697, Accuracy: 0.9141, Time: 213.83 seconds
Epoch 10, Loss: 0.2460, Accuracy: 0.9228, Time: 213.28 seconds
Epoch 11, Loss: 0.2250, Accuracy: 0.9300, Time: 212.39 seconds
Epoch 12, Loss: 0.2070, Accuracy: 0.9353, Time: 213.99 seconds
Epoch 13, Loss: 0.1944, Accuracy: 0.9389, Time: 214.26 seconds
Epoch 14, Loss: 0.1728, Accuracy: 0.9455, Time: 213.34 seconds
Epoch 15, Loss: 0.1682, Accuracy: 0.9484, Time: 212.97 seconds
Epoch 16, Loss: 0.1634, Accuracy: 